In [ ]:
# import the necessary packages
import requests
url = 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
r = requests.get(url, allow_redirects=True)
open('nature_12K.zip', 'wb').write(r.content)
!wget 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
!unzip nature_12K.zip

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.init(project="CS6910_Assignment2", entity="swe-rana")
from wandb.keras import WandbCallback

wandb: Currently logged in as: swe-rana (use `wandb login --relogin` to force relogin)


In [ ]:
import pandas as pd
import os
import keras
import tensorflow as tf
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def data(image_size,augment_data,batch_size):
      home_path = "/content/inaturalist_12K" # initialize the path to the *original* input directory of images
      # derive the paths to the training, validation, and testing
      train_path = os.path.join(home_path,'train')
      test_path = os.path.join(home_path,'val')
      if augment_data == True:        # initialize the training data augmentation object
        train_data = ImageDataGenerator(rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True,brightness_range=[0.2,1.2],validation_split=0.1) # brightness
        test_data = ImageDataGenerator(rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True,brightness_range=[0.2,1.2]) # brightness

      else:
        
        train_data = ImageDataGenerator(rescale=1./255, validation_split=0.1) # initialize the training generator
        test_data = ImageDataGenerator(rescale=1./255) # initialize the test generator


      train_generator = train_data.flow_from_directory(directory= train_path,target_size=(image_size, image_size), batch_size=batch_size,color_mode="rgb",class_mode="categorical",subset = "training")
      val_generator = train_data.flow_from_directory(directory= train_path,target_size=(image_size, image_size), batch_size=batch_size,color_mode="rgb",class_mode="categorical",subset = "validation")
      test_generator = test_data.flow_from_directory(directory= test_path,target_size=(image_size, image_size), batch_size=batch_size,color_mode="rgb",class_mode="categorical")
      return train_generator,val_generator,test_generator

def pretrained_model(image_size,pre_train_mod,dropout,epochs,dense_size,lr):
           # the base model
           #pre-training
            if pre_train_mod =="InceptionV3":
                        base_model = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet',input_shape=(image_size, image_size, 3), include_top=False)
            if pre_train_mod == "ResNet50":
                        base_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet',input_shape=(image_size, image_size, 3), include_top=False)
            if pre_train_mod == "InceptionResNetV2":
                        base_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet',input_shape=(image_size, image_size, 3), include_top=False)
            if pre_train_mod == "Xception":
                        base_model =  tf.keras.applications.xception.Xception (weights='imagenet',input_shape=(image_size, image_size, 3), include_top=False)
            #fine tuning
            # loop over all layers in the base model and freeze them so they will not be updated during the first training process
            for layer in base_model.layers:
                         layer.trainable = False
            # construct the head of the model that will be placed on top of  the base model           
            x = layers.Flatten()(base_model.output)            
            x = Dense(dense_size, activation='relu')(x)
            x=tf.keras.layers.Dropout(dropout)(x)
            predictions = Dense(10, activation='softmax')(x)
            # this is the model we will train
            model = Model(inputs=base_model.input, outputs=predictions)# place the head FC model on top of the base model
            
            return model


  


In [ ]:
def setRunName(image_size,pre_train_mod, dropout,augment_data,batch_size,dense_size):
    
    augment_data_options = {True: "Y", False: "N"}

    run_name = "_".join(["img",str(image_size),"pre",pre_train_mod,"drop", str(dropout), "bat",str(batch_size),"den",str(dense_size)])
    
    return run_name


In [ ]:
    def train():

        config_defaults = {
            "image_size" :512,
            "pre_train_mod" : "InceptionResnetV2",
            "augment_data": False,
            "dropout": 0.2,
            "epochs": 10,
            "batch_size" : 256,
            "dense_size": 64,
            "lr": 0.001
        }
        wandb.init(config=config_defaults)
        config = wandb.config
        wandb.run.name = setRunName(config.image_size,config.pre_train_mod, config.augment_data, config.dropout,config.batch_size,config.dense_size)
       
        train_gen, val_gen, test_gen = data(image_size=config.image_size,augment_data=config.augment_data,batch_size=config.batch_size) #generating train,test and val data
        model = pretrained_model(image_size=config.image_size,pre_train_mod=config.pre_train_mod,dropout=config.dropout,epochs=config.epochs,dense_size=config.dense_size,lr=config.lr)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="categorical_accuracy")# compile our model
        model.fit(train_gen, epochs=config.epochs, validation_data=val_gen, callbacks=[WandbCallback()])#fitting our model into train and val data


In [ ]:
#Set up a sweep config
sweep_config = {
    "name": "Assignment_2_partB_all_params",
    "description": "Assignment_B_partA_all_params",
    "method": "bayes",
    "metric" : {
        "name" : "val_categorical_accuracy",
        "goal" : "maximize"
    },
    "project": "CS6910_Assignment2",
    "parameters": {
        "pre_train_mod" :{
            "values" : ["Resnet50","InceptionV3","InceptionResNetV2","Xception"]
        },
        "image_size" : {
            "values" :[200,256,512]
        },
        "batch_size":{
            "values":[200,256,400]
        },
        "dropout": {
            "values": [0.2,0.3]
        },
        "augment_data": {
            "values": [True,False]
        },
        "dense_size": {
            "values": [32,64]
        },
        "lr": {
            "values": [0.001]
        },
        "epochs": {
            "values": [10]
        }
    }

}
# creating the sweep


In [ ]:
sweep_id = "g89b0365"
#sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id,train,entity="swe-rana",project="CS6910_Assignment2")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: mufhb5n4 with config:
wandb: 	augment_data: True
wandb: 	batch_size: 200
wandb: 	dense_size: 32
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	image_size: 200
wandb: 	lr: 0.001
wandb: 	pre_train_mod: Xception


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
45/45 [==============================] - 250s 5s/step - loss: 5.6053 - categorical_accuracy: 0.1014 - val_loss: 2.3026 - val_categorical_accuracy: 0.1001 - _timestamp: 1648463720.0000 - _runtime: 264.0000
Epoch 2/10
45/45 [==============================] - 211s 5s/step - loss: 2.3026 - categorical_accuracy: 0.1000 - val_loss: 2.3026 - val_categorical_accuracy: 0.1001 - _timestamp: 1648463931.0000 - _runtime: 475.0000
Epoch 3/10
45/45 [==============================] - 206s 5s/step - loss: 2.3026 - categorical_accuracy: 0.1000 - val_loss: 2.3026 - val_categorical_accuracy: 0.1001 - _timestamp: 1648464138.0000 - _runtime: 682.0000
Epoch 4/10
45/45 [==============================] - 206s 5s/step - loss: 2.3028 - categorical_accuracy: 0.0962 - val_loss: 2.3026 - val_categorical_accuracy: 0.0991 - _timestamp: 1648464344.0000 - _runtime: 888.0000
Epoch 5/

categorical_accuracy,█▇▇▄▅▇▂▁▄▄
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
val_categorical_accuracy,███▁██████
val_loss,█▅▆▇▄▅▁▃▃▁
best_epoch,6
best_val_loss,2.30258
categorical_accuracy,0.09711
epoch,9
loss,2.30265
val_categorical_accuracy,0.1001


wandb: Agent Starting Run: av51mo5x with config:
wandb: 	augment_data: False
wandb: 	batch_size: 200
wandb: 	dense_size: 64
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	image_size: 512
wandb: 	lr: 0.001
wandb: 	pre_train_mod: InceptionResNetV2


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
45/45 [==============================] - 392s 8s/step - loss: 2.3509 - categorical_accuracy: 0.4808 - val_loss: 0.9688 - val_categorical_accuracy: 0.7227 - _timestamp: 1648466057.0000 - _runtime: 404.0000
Epoch 2/10
45/45 [==============================] - 344s 8s/step - loss: 0.9678 - categorical_accuracy: 0.6917 - val_loss: 0.7696 - val_categorical_accuracy: 0.7928 - _timestamp: 1648466401.0000 - _runtime: 748.0000
Epoch 3/10
45/45 [==============================] - 344s 8s/step - loss: 0.7933 - categorical_accuracy: 0.7619 - val_loss: 0.6682 - val_categorical_accuracy: 0.8388 - _timestamp: 1648466746.0000 - _runtime: 1093.0000
Epoch 4/10
45/45 [==============================] - 342s 8s/step - loss: 0.6395 - categorical_accuracy: 0.7950 - val_loss: 0.6979 - val_categorical_accuracy: 0.8288 - _timestamp: 1648467091.0000 - _runtime: 1438.0000
Epoch 

categorical_accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
val_categorical_accuracy,▁▅▇▇▇█▇█▇█
val_loss,█▃▁▂▂▃▂▂▁▁
best_epoch,9
best_val_loss,0.66478
categorical_accuracy,0.85967
epoch,9
loss,0.42796
val_categorical_accuracy,0.85185


wandb: Agent Starting Run: t51kwscb with config:
wandb: 	augment_data: True
wandb: 	batch_size: 200
wandb: 	dense_size: 32
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	image_size: 256
wandb: 	lr: 0.001
wandb: 	pre_train_mod: InceptionV3


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
45/45 [==============================] - 277s 6s/step - loss: 11.8925 - categorical_accuracy: 0.0997 - val_loss: 2.3026 - val_categorical_accuracy: 0.1001 - _timestamp: 1648469517.0000 - _runtime: 293.0000
Epoch 2/10
45/45 [==============================] - 260s 6s/step - loss: 2.3026 - categorical_accuracy: 0.0962 - val_loss: 2.3026 - val_categorical_accuracy: 0.1001 - _timestamp: 1648469777.0000 - _runtime: 553.0000
Epoch 3/10
45/45 [==============================] - 257s 6s/step - loss: 2.3028 - categorical_accuracy: 0.0973 - val_loss: 2.3026 - val_categorical_accuracy: 0.1001 - _timestamp: 1648470034.0000 - _runtime: 810.0000
Epoch 4/10
24/45 [===============>..............] - ETA: 1:47 - loss: 2.3026 - categorical_accuracy: 0.1004